In [7]:
#IMPORTS
import os,urllib.request

# PROVIDE YOUR DOWNLOAD DIRECTORY HERE
datapath = '../../Data/MNISTData/'

# CREATING DOWNLOAD DIRECTORY
if not os.path.exists(datapath):
    os.makedirs(datapath)

# URLS TO DOWNLOAD FROM
urls = ['http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
       'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
       'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
       'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz']

for url in urls:
    filename = url.split('/')[-1]   # GET FILENAME
    
    if os.path.exists(datapath+filename):
        print(filename, ' already exists')  # CHECK IF FILE EXISTS
    else:
        print('Downloading ',filename)
        urllib.request.urlretrieve (url, datapath+filename) # DOWNLOAD FILE
     
print('All files are available')

All files are available


In [8]:
import os,gzip,shutil

# PROVIDE YOUR DOWNLOAD DIRECTORY HERE
datapath = '../../Data/MNISTData/'  

# LISTING ALL ARCHIVES IN THE DIRECTORY
files = os.listdir(datapath)
for file in files:
    if file.endswith('gz'):
        print('Extracting ',file)
        with gzip.open(datapath+file, 'rb') as f_in:
            with open(datapath+file.split('.')[0], 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
print('Extraction Complete')

# OPTIONAL REMOVE THE ARCHIVES
for file in files:
    print('Removing ',file)
    os.remove(datapath+file)
print ('All archives removed')

Extracting  t10k-images-idx3-ubyte.gz
Extracting  train-images-idx3-ubyte.gz
Extracting  train-labels-idx1-ubyte.gz
Extracting  t10k-labels-idx1-ubyte.gz
Extraction Complete
Removing  t10k-images-idx3-ubyte.gz
Removing  train-images-idx3-ubyte.gz
Removing  train-labels-idx1-ubyte.gz
Removing  t10k-labels-idx1-ubyte.gz
All archives removed


In [2]:
import os,codecs,numpy

# PROVIDE YOUR DIRECTORY WITH THE EXTRACTED FILES HERE
datapath = '../../Data/MNISTData/'

files = os.listdir(datapath)

def get_int(b):   # CONVERTS 4 BYTES TO A INT
    return int(codecs.encode(b, 'hex'), 16)

data_dict = {}
for file in files:
    if file.endswith('ubyte'):  # FOR ALL 'ubyte' FILES
        print('Reading ',file)
        with open (datapath+file,'rb') as f:
            data = f.read()
            type = get_int(data[:4])   # 0-3: THE MAGIC NUMBER TO WHETHER IMAGE OR LABEL
            length = get_int(data[4:8])  # 4-7: LENGTH OF THE ARRAY  (DIMENSION 0)
            if (type == 2051):
                category = 'images'
                num_rows = get_int(data[8:12])  # NUMBER OF ROWS  (DIMENSION 1)
                num_cols = get_int(data[12:16])  # NUMBER OF COLUMNS  (DIMENSION 2)
                parsed = numpy.frombuffer(data,dtype = numpy.uint8, offset = 16)  # READ THE PIXEL VALUES AS INTEGERS
                parsed = parsed.reshape(length,num_rows,num_cols)  # RESHAPE THE ARRAY AS [NO_OF_SAMPLES x HEIGHT x WIDTH]           
            elif(type == 2049):
                category = 'labels'
                parsed = numpy.frombuffer(data, dtype=numpy.uint8, offset=8) # READ THE LABEL VALUES AS INTEGERS
                parsed = parsed.reshape(length)  # RESHAPE THE ARRAY AS [NO_OF_SAMPLES]                           
            if (length==10000):
                set = 'test'
            elif (length==60000):
                set = 'train'
            data_dict[set+'_'+category] = parsed  # SAVE THE NUMPY ARRAY TO A CORRESPONDING KEY

Reading  t10k-images-idx3-ubyte
Reading  t10k-labels-idx1-ubyte
Reading  train-images-idx3-ubyte
Reading  train-labels-idx1-ubyte


In [13]:
train_images = data_dict['train_images']
print(train_images.shape)
train_labels = data_dict['train_labels']
print(train_labels.shape)
test_images = data_dict['test_images']
print(test_images.shape)
test_labels = data_dict['test_labels']
print(test_labels.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [36]:
import numpy as np
import array

# Энд train_images гэсэн энгийн array - г numpy-ын array болгож байна. 

np.array(train_images)
np.array(train_labels)
np.array(test_images)
np.array(test_labels)

class Neural_network:
    def __init__(train_image_size,picture_size,hidden_layer_size,output_layer_size,batch_size):
        # Divisor нь 0-1 н хооронд болгохын тулд үржиж байгаа тоо.
        divisor = 1/255
        # Харин энд зураг тус бүрийг 0-1 хооронд байхаар pixel бүрээр үржиж байна гэсэн үг.
        c_train_image = train_images*(divisor)
        c_test_image = test_images*(divisor)
        
        print("train image shape:")
        print(c_train_image.shape)
        print("test image shape:")
        print(c_test_image.shape)
        
        # Энд (60000,28,28) хэмжээтэй зураг аа (60000,784) хэмжээтэй болгож байна.
        cv_train_image = np.reshape(c_train_image,(60000,784))
        print("Сургах өгөгдлөө 2 хэмжээст болгож байна.")
        print(cv_train_image.shape)
        cv_test_image = np.reshape(c_test_image,(10000,784))
        print("Турших өгөгдлөө 2 хэмжээст болгож байна.")
        print(cv_test_image.shape)
    
    def generate_random(self,picture_size,hidden_layer_size,output_layer_size):
        w1 = np.random.randn(picture_size,hidden_layer_size)*0.01
        b1 = np.zeros((1,hidden_layer_size))
        w2 = np.random.randn(hidden_layer_size,output_layer_size)*0.01
        b2 = np.zeros((1,output_layer_size))
        return w1,w2,b1,b2
    
    def softmax(self,x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0) # only difference
    
    def relu(self,x):
        return np.maximum(0,x)
        
    def compute_loss(self,w1,b1,w2,b2):
        batch_size = 100
        loss = 0
        for index in range(0,1000,batch_size):
            batch=cv_train_image[index:min(index+batch_size,cv_train_image.shape[0]),:]
            layer1 = np.add(batch@w1,b1)
            layer1 = relu(layer1)
            layer2 = np.add(layer1@w2,b2)
            layer2 = softmax(layer2)
            for i in range(batch_size):
                correct_logprobs = -np.log(layer2[i,train_labels[i]])
                loss = loss + correct_logprobs
#                 print(i)
#                 print("Correct:",correct_logprobs)
            loss = loss / batch_size
            print("Loss:",loss)
        return loss        
# Сургах зурагны тоо
train_image_size = 60000
picture_size = 784
hidden_layer_size = 32
output_layer_size = 10
batch_size = 100
print("Starting ...")
Object1 = Neural_network(train_image_size,picture_size,hidden_layer_size,output_layer_size)
w1,w2,b1,b2 = Object1.generate_random(picture_size,hidden_layer_size,output_layer_size)
loss = Object1.compute_loss(w1,b1,w2,b2)
print(loss)

# def softmax(inputs):
#     return np.exp(inputs) / float(sum(np.exp(inputs)))
    

Starting ...
train image shape:
(60000, 28, 28)
test image shape:
(10000, 28, 28)
Сургах өгөгдлөө 2 хэмжээст болгож байна.
(60000, 784)
Турших өгөгдлөө 2 хэмжээст болгож байна.
(10000, 784)
Loss: 4.604725245638579
Loss: 4.6512882493836845
Loss: 4.6523937052444975
Loss: 4.651995789185555
Loss: 4.651839628563409
Loss: 4.651179493929227
Loss: 4.6513993088887995
Loss: 4.651408934243461
Loss: 4.651249784620968
Loss: 4.6519499539421005
4.6519499539421005
